# LLMs and CSS

In this short notebook, we'll see how LLMs can be used to annotate and generate data for studying linguistic constructs in text.

Let's break down the goals:

* **Goal 1:** Learn langchain.
    * Langchain is the most common library for interacting with LLMs in Python!

* **Goal 2:** Annotate a dataset with ChatGPT
    * We'll annotate texts as being sarcastic or not using ChatGPT.
* **Goal 3:** Generate Synthetic Data for a Specific Construct on Interest
    * We'll generate new examples of sarcastic and non-sarcastic texts.
    
A few requirements.
1. You will need an OpenAI key to generate the data. Since the data has already been generated, you won't need it to explore the synthetic data, but if you want to re-run the generation you will need to get a key. You can signup [here](https://openai.com/blog/openai-api)
2. *(If you have an API key)* In the .env file in root add your API key.
3. Run the requirements.txt file to pip install all the necessary libraries.

### Local Setup
Let's install all the required libraries to go through this document. 

In [ ]:
requirements = "requirements.txt"
!pip install -r {requirements}

In [1]:
import pandas as pd
import langchain
from dotenv import load_dotenv
import os

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from utils import *

load_dotenv()  # take environment variables from .env.

True

Run this cell to load the autoreload extension. This allows us to edit .py source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
seed = 42    # for reproducibility 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 1. Dataset Introduction

The dataset includes two columns: `text` and `labels`. Where `text` is a Tweet and `label` is either sarcastic or non-sarcastic.

In [4]:
df = pd.read_json("data/sarcasm.json", orient="records")

In [5]:
print("Number of rows: ",len(df))
print("Number of sarcastic comments: ",len(df[df["labels"]=="sarcastic"]))
print()
example_rows(df)

Number of rows:  500
Number of sarcastic comments:  135

Example of a sarcastic text
do people with clear skin feel accomplished?? superior??? comfortable in their own skin???? whats that like lmfao

Example of a non-sarcastic text
A message to all Muslims and Refugees: I'm sorry for how my country is treating you. You are only human. #RefugeesDetained #Trump #rt


## 2. Langchain
> [LangChain](https://python.langchain.com/docs/get_started/quickstart) is a framework for developing applications powered by language models. 

We will use Langchain to *annotate* and *generate* sarcastic texts! Langchain is currently the most widely used Python library for interacting with these LLMs programatically. It opens up a lot of cool functionalities, but we will limit to a simple case: given a prompt, let's generate text!

To design prompts we need to add both a `system` prompt and a `message` prompt. In Langchain this corresponds `HumanMessagePromptTemplate` and `SytemMessagePromptTemplate`. To read more about prompt templates you can look at the Langchain documentation [here](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/).

All the prompts are included in the `utils.py` file but we will add an example below.

The **system** message basically puts the model into a certain headspace through meta-instructions. E.g., "You are a helpful assistant!".

The **human message** instead includes the actual task explanation. 

In this code, we ask the model to generate `{num_generations}` (for example 10) `{direction}` (for example sarcastic) comments. 

The function will then return a list with the two messages which we will feed into Langchain's LLM. :)


```py
def sarcasm_simple_prompt(self) -> list:
    system_message = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],
            template="You are a model that generates sarcastic and non-sarcastic texts."
        )
    )
    human_message = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["num_generations", "direction"],
            template="Generate {num_generations} {direction} texts. Ensure diversity in the generated texts."
        )
    )
    return [system_message, human_message]
```





### 2.1 Annotations
Let's annotate texts as being sarcastic or not, and reporting the performance!

First, let's try it on one text:
> do people with clear skin feel accomplished?? superior??? comfortable in their own skin???? whats that like lmfao

*Only run the code if you have an OpenAI key, otherwise just import the files with already generated data.*

In [35]:
example_text = "do people with clear skin feel accomplished?? superior??? comfortable in their own skin???? whats that like lmfao"

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)
prompt = ChatPromptTemplate.from_messages(sarcasm_annotate_prompt())
chain = LLMChain(prompt=prompt, llm=llm)
generated = chain.run({"text": example_text})

In [9]:
print(generated)

Sarcastic


Great! It seems to work. Now we will iterate through all the sarcastic texts in our document.

In [ ]:
generated = []
for i, row in df.iterrows():
    text = row["text"]
    prompt = ChatPromptTemplate.from_messages(sarcasm_annotate_prompt())
    chain = LLMChain(prompt=prompt, llm=llm)
    generated.append(chain.run({"text": example_text}))
    
df["predict"] = generated

The annotations have already been run, so let's just import the dataset.

In [24]:
df = pd.read_json("data/annotate_gpt-3.5-turbo.json")

In [25]:
def process_text(x):
    """
    Process GPT outputs. Otherwise 
    """
    if "non-sarcastic" in x.lower():
        return "not-sarcastic"
    else:
        return "sarcastic"
    
df["predict"] = df["predict"].apply(lambda x: process_text(x))


Let's import some metrics to see how well the predictions are.

In [34]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(df["target"], df["predict"])
precision, recall, f1, _ = precision_recall_fscore_support(df["target"], df["predict"], average="macro")

print(f"Accuracy: {accuracy}")
print(f"F1 score: {round(f1, 3)}")

Accuracy: 0.602
F1 score: 0.596


Not amazing!

Instead, we can try to generate more data. 

### 2.2 Generating data
Now we'll quickly go over how to generate more sarcastic texts. This can be used for *de-novo* dataset creation or for data augmentation. 

We'll use a grounded prompting technique, where we'll rewrite real tweets to make them sarcastic or not!

Let's rewrite this Tweet as an example:
> Tapping a tuning fork and seeing who resonates

In [36]:
example_text = "Tapping a tuning fork and seeing who resonates"

In [47]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9, max_tokens=512)
prompt = ChatPromptTemplate.from_messages(sarcasm_grounded_prompt())
chain = LLMChain(prompt=prompt, llm=llm)
generated = chain.run({"text": example_text, "direction": "sarcastic", "num_generations": 1})

In [48]:
print(generated)

'Oh yeah, because tapping a tuning fork and seeing who resonates is clearly the pinnacle of intellectual pursuits. '

Don't run the following code if OpenAI key not connected, just import csv!

In [ ]:
generated = []
for i, row in df.iterrows():
    for direction in ["sarcastic", "not-sarcastic"]:
        text = row["text"]
        prompt = ChatPromptTemplate.from_messages(sarcasm_grounded_prompt())
        chain = LLMChain(prompt=prompt, llm=llm)
        generated.append(chain.run({"text": example_text, "direction": direction}))
    
df["augmented_text"] = generated

In [49]:
df = pd.read_json("data/grounded_gpt-3.5-turbo.json")

## Analysis

Let's see, if there are any ideosyncracies in the generated sarcastic texts!

There is a lot that can be done here, but we will look at the prevelance of "Oh" in sarcastic comments between the two groups.

In [54]:
generated_sarcastic = df[df["labels"]=="sarcastic"]["augmented_text"].values
original_sarcastic = df[df["target"]=="sarcastic"].drop_duplicates(subset="text")["text"].values

In [58]:
oh_synthetic = len([k for k in generated_sarcastic if "oh" in k.lower()]) / len(generated_sarcastic)
oh_real = len([k for k in original_sarcastic if "oh" in k.lower()]) / len(original_sarcastic)

print(f"'Oh' present in {round(oh_synthetic, 3)} of synthetic texts")
print()
print(f"'Oh' present in {round(oh_real, 3)} of real texts")

'Oh' present in 0.165 of synthetic texts

'Oh' present in 0.022 of real texts
